In [1]:
import warnings
warnings.simplefilter(action='ignore')
import pandas as pd
import numpy as np
import os
import glob2
import re
#import progressbar
import tqdm
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# path = r"C:\4bc_office_laptop\Nilesh\GBC_Project\GBC_Maf\S"
# path = r"C:\4bc_office_laptop\Nilesh\GBC_Project\GBC_Maf\S\S_CustFilt"
# path = r"C:\4bc_office_laptop\Nilesh\GBC_Project\GBC_Maf\G"
# path = r"C:\4bc_office_laptop\Nilesh\GBC_Project\DNA\GERMLINE\Gmam\G_CustFilt"
# path = r"C:\4bc_office_laptop\Nilesh\GBC_Project\DNA\SOMATIC\Smam\S_CustFilt"
path = r"C:\4bc_office_laptop\Nilesh\GBC_Project\DNA\SOMATIC\Smam\S_CustFilt_new"
# path = r"C:\4bc_office_laptop\Nilesh\GBC_Project\DNA\SOMATIC\Smam"

In [3]:
files= glob2.glob(os.path.join(path, "*.csv"))
# ce_files = []
# abs_files =[]

# for i in files:
#   if '-SE8-' in i or '-SSE-' in i or 'SSE' in i or 'SE8' in i or '-B-' in i:
#     ce_files.append(i)
#   else:
#     abs_files.append(i)

In [4]:
# ce_files = abs_files
ce_files = files

In [ ]:
ce_files

In [ ]:
for i in tqdm.tqdm(range(len(ce_files))):
  df = pd.read_csv(ce_files[i]).drop_duplicates('IGV_link')
  sp1=re.split(r'/|\\',ce_files[i])[-1]
  col_n=sp1.split('_')[0]
  df[f'{col_n}:DP'] = df[f'{col_n}:DP'].replace('.', '0').astype('float')
  df = df[df['FILTER']=='PASS']

  ## Pop frequency
  columns_to_process = ['esp6500siv2_all', 'ExAC_ALL', 'ExAC_SAS', 'AF', 'AF_sas', '1000g2015aug_all', '1000g2015aug_all', '1000g2015aug_SAS', 'Ref_Depth']

  for column in columns_to_process:
    df[column] = df[column].replace('.', '0').astype('float')

  df = df[df['esp6500siv2_all']<= 0.01]
  df = df[df['ExAC_ALL']<= 0.01]
  df = df[df['ExAC_SAS']<= 0.01]
  df = df[df['AF']<=0.01]
  df = df[df['AF_sas']<= 0.01]
  df = df[df['1000g2015aug_all']<= 0.01]
  df = df[df['1000g2015aug_all']<= 0.01]
  df = df[df['1000g2015aug_SAS']<= 0.01]
  # df = df[df['Ref_Depth']>=2]
  df = df[df['ExonicFunc.ensGene']!='synonymous SNV']
  df = df[~df['InterVar_automated'].str.contains('benign', case=False)]
  df.to_csv(fr'C:\4bc_office_laptop\Nilesh\GBC_Project\DNA\SOMATIC\Smam\S_CustFilt_new\{col_n}_filtered.csv', index=False)


## Compose the binary file

In [ ]:
my_df=[]



for i in tqdm.tqdm(range(len(ce_files))):
  df = pd.read_csv(ce_files[i]).drop_duplicates()
  sp1=re.split(r'/|\\',ce_files[i])[-1]
  col_n=sp1.split('_')[0]
  row_list=df['Ref.Gene'].tolist()
  col_list=df['InterVar_automated'].tolist()
  new_df=pd.DataFrame(col_list,index=row_list,columns=[col_n])
  new_df=new_df.replace("Uncertain_significance","1")
  new_df=new_df.replace("Uncertain_significance","1")
  new_df=new_df.replace("Benign","2")
  new_df=new_df.replace("Likely_benign","3")
  new_df=new_df.replace("Likely_pathogenic","5")
  new_df=new_df.replace('Pathogenic',"6")
  new_df=new_df.replace('.','4')
  new_df=new_df.astype(int)
  mod_df = new_df.copy()
  mod_df[new_df > 4] = 1 # Pathogenic
  mod_df[(new_df <= 4) & (new_df > 0)] = 1 #Benign
  mod_df.fillna(0, inplace=True)
  mod_df = mod_df.groupby(mod_df.index.tolist(),axis=0)
  grouped_df=mod_df.sum()
  my_df.append(grouped_df)

In [ ]:
fdf = pd.concat(my_df, axis=1).fillna(0).T
fdf.sum().sum()

In [8]:
fdf = pd.concat(my_df, axis=1).fillna(0).T
fdf = fdf.apply(lambda x: x.apply(lambda y: 1 if y > 0 else 0))

def fbc_rename(x):
  return 'IN-423-'+x.split('-')[0]
new_indexes = []
for i in fdf.index.to_list():
  new_indexes.append(fbc_rename(i))

fdf.set_index(pd.Index(new_indexes), inplace=True)

fdf.drop_duplicates(inplace=True)

In [ ]:
fdf

In [10]:
freq_df = fdf.sum(axis=0).reset_index()
freq_df.columns  = ['Gene', 'Frequency']

In [11]:
freq_df.sort_values('Frequency', ascending=False).to_excel('GBC_S_new_v2_gene_freq.xlsx')

In [ ]:
plt.figure(figsize=(40, 8))
custom_bw_palette = ['#FFFFFF','#000000']

# Set the custom palette in Seaborn
x=sns.color_palette(custom_bw_palette)

sns.heatmap(fdf[freq_df.sort_values('Frequency', ascending=False).head(100)['Gene'].to_list()], cmap=x, linewidths=1, linecolor='black')


In [ ]:
freq_df.sort_values('Frequency', ascending=False).head(100)['Gene'].to_list()

In [ ]:
plt.figure(figsize=(40, 8))
sns.barplot(freq_df.sort_values('Frequency', ascending=False).head(100), x='Gene', y='Frequency')
var = plt.xticks(rotation=90)

In [ ]:
## Gene frequency within Dataset

result_dict = {}
for index, row in fdf.iterrows():
    genes_list = row[row == 1].index.tolist()
    result_dict[index] = genes_list

gene_frequency = {}
for genes_list in result_dict.values():
    for gene in genes_list:
        if gene in gene_frequency:
            gene_frequency[gene] += 1
        else:
            gene_frequency[gene] = 1

gf =pd.DataFrame(list(gene_frequency.items()), columns=['Gene', 'Frequency']).sort_values('Frequency', ascending=False).iloc[:100, :]
gf.to_csv('gene_freq_WC_cancer.csv')
plt.figure(figsize=(30, 5))
sns.barplot(gf, x='Gene', y='Frequency')
var = plt.xticks(rotation=90)